# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [13]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [14]:
# Create a temporary workspace
import tempfile


SESSION_WORKSPACE = tempfile.mkdtemp()
print(f"Session workspace created at: {SESSION_WORKSPACE}")

Session workspace created at: /tmp/tmp0xdxo_jl


In [15]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Unzip the dataset
import shutil
import os


DATASET_DIR_NAME = "duckietown_object_detection_dataset"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

Unpacking zipped data...
Zipped dataset unpacked to /tmp/tmp0xdxo_jl/duckietown_object_detection_dataset
#Files in train/images: 1604
#Files in train/labels: 1604
#Files in val/images: 402
#Files in val/labels: 402


In [17]:
# Change  working directory to the session workspace
os.chdir(SESSION_WORKSPACE)
print(f"PWD: {os.getcwd()}")

PWD: /tmp/tmp0xdxo_jl


# Next, we will clone Yolov5

In [18]:
!rm -rf ./yolov5
!git clone https://github.com/duckietown/yolov5.git -b dt-obj-det
#!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!cd yolov5 && pip3 install -r requirements.txt
!pip3 install torch==1.7.0 torchvision==0.8.1

Cloning into 'yolov5'...
remote: Enumerating objects: 6174, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 6174 (delta 5), reused 0 (delta 0), pack-reused 6163
Receiving objects: 100% (6174/6174), 8.44 MiB | 12.16 MiB/s, done.
Resolving deltas: 100% (4215/4215), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# We can now inform the training process of the format and location of our dataset

In [19]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../duckietown_object_detection_dataset/train
val: ../duckietown_object_detection_dataset/val

# number of classes
nc: 4

# class names
names: [ 'duckie', 'cone', 'truck', 'bus' ]

Overwriting ./yolov5/data/duckietown.yaml


# And we're ready to train! This step will take about 5 minutes.

Notice that we're only training for 10 epochs. That's probably not enough!

In [20]:
!cd yolov5 && git pull && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 64 --epochs 1 --data duckietown.yaml --weights yolov5s.pt

Already up to date.
github: up to date with https://github.com/duckietown/yolov5 ✅
YOLOv5 🚀 v5.0-71-g0343bc7 torch 1.7.0 CPU

Namespace(adam=False, artifact_alias='latest', batch_size=64, bbox_interval=-1, bucket='', cache_images=False, cfg='./models/yolov5n.yaml', data='./data/duckietown.yaml', device='', entity=None, epochs=1, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=64, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momen

In [21]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
# print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

None
Marked the model from the latest run (exp) as yolov5/best.pt.


# Next, we can upload your model to Duckietown's cloud!

In [22]:
!pip3 install git+https://github.com/duckietown/lib-dt-mooc@mooc2022

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/duckietown/lib-dt-mooc (to revision mooc2022) to /tmp/pip-req-build-9kece66r
  Running command git clone -q https://github.com/duckietown/lib-dt-mooc /tmp/pip-req-build-9kece66r
  Running command git checkout -b mooc2022 --track origin/mooc2022
  Switched to a new branch 'mooc2022'
  Branch 'mooc2022' set up to track remote branch 'mooc2022' from 'origin'.
     |████████████████████████████████| 142 kB 4.2 MB/s 
  Created wheel for dt-mooc: filename=dt_mooc-0.0.5-py3-none-any.whl size=6489 sha256=6d6af220265565e2de07106753a43dd8924e8fa618da63b233c66fb4e6968ff4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ggzwfhet/wheels/7a/2e/72/79fb182d351314e4d6fb9aff8bdd01c1610c981db999440152
  Created wheel for dt-data-api-daffy: filename=dt_data_api_daffy-1.1.3-py3-none-any.whl size=13714 sha256=0c25cff3bde96ab7a0645d8250007dc6f1e2bc8524c9d0876f5ee93ea9199560
  Store

In [23]:
# TODO: Fill in the duckietown token here
YOUR_DT_TOKEN = "dt1-daZUHiuSz7CV52v4YS9wpXWsgnVtPyMP4sFd39x53WeZa-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfiQFRKqBYf6sPsgu6ELuBXhtVPzwZJqnAm"

In [24]:
from dt_mooc.cloud import Storage
from dt_mooc.utils import select_device
import sys
import torch
from pathlib import Path

sys.path.insert(0, './yolov5')

destination_name = "yolov5mnfp32"
best = torch.load("./yolov5/best.pt", map_location=select_device("cpu"))
pt_model = best['model']
pt_weights_path = "./yolov5/best.pt"
storage = Storage(YOUR_DT_TOKEN)


# convert to fp32
device = select_device('cpu')
pt_model = pt_model.to(device).float()  # load to FP32
pt_model.eval()

best['model'] = pt_model

# save to disk
pt_weights_path = os.path.join(os.path.dirname(pt_weights_path), f"{destination_name}.pt")
torch.save(best, pt_weights_path)
print(pt_weights_path)

# compute hash
storage.hash(pt_weights_path)
hash_path = pt_weights_path+".sha256"

# UPLOAD .pt
storage._upload(destination_name, [pt_weights_path, hash_path])

./yolov5/yolov5mnfp32.pt
Uploading file `yolov5mnfp32.pt`...

File `yolov5mnfp32.pt` successfully uploaded! It will now be found at `courses/mooc/objdet/data/nn_models/yolov5mnfp32.pt`.
Uploading file `yolov5mnfp32.pt.sha256`...

File `yolov5mnfp32.pt.sha256` successfully uploaded! It will now be found at `courses/mooc/objdet/data/nn_models/yolov5mnfp32.sha256`.


In [24]:
from dt_mooc.cloud import Storage
from dt_mooc.utils import select_device
import sys
import torch
from pathlib import Path

sys.path.insert(0, './yolov5')

destination_name = "yolov5hf"
best = torch.load("./yolov5/best.pt", map_location=select_device("cpu"))
pt_model = best['model']
pt_weights_path = "./yolov5/best.pt"
storage = Storage(YOUR_DT_TOKEN)


# convert to fp16
device = select_device('cpu')
pt_model = pt_model.to(device).half()  # load to FP16
pt_model.eval()

best['model'] = pt_model

# save to disk
pt_weights_path = os.path.join(os.path.dirname(pt_weights_path), f"{destination_name}.pt")
torch.save(best, pt_weights_path)
print(pt_weights_path)

# compute hash
storage.hash(pt_weights_path)
hash_path = pt_weights_path+".sha256"

# UPLOAD .pt
storage._upload(destination_name, [pt_weights_path, hash_path])

./yolov5/yolov5hf.pt
Uploading file `yolov5hf.pt`...



File `yolov5hf.pt` successfully uploaded! It will now be found at `courses/mooc/objdet/data/nn_models/yolov5hf.pt`.
Uploading file `yolov5hf.pt.sha256`...

File `yolov5hf.pt.sha256` successfully uploaded! It will now be found at `courses/mooc/objdet/data/nn_models/yolov5hf.sha256`.


In [15]:
from dt_mooc.cloud import Storage
from dt_mooc.utils import select_device
import sys
import torch

storage = Storage(YOUR_DT_TOKEN)

storage.upload_yolov5_only("yolov5", model, "./yolov5/best.pt")

Uploading file `best.pt`...



File `best.pt` successfully uploaded! It will now be found at `courses/mooc/objdet/data/nn_models/yolov5.pt`.
Uploading file `best.pt.sha256`...

File `best.pt.sha256` successfully uploaded! It will now be found at `courses/mooc/objdet/data/nn_models/yolov5.sha256`.


# Done!

We're done training! You can now close this tab and go back to the `Training` notebook